# SORA - Stellar Occultation Reduction and Analysis

## Ephemeris Object Classes

The Ephemeris Classes within SORA were created to generate geocentric ephemeris for an object. The documentation at <font color=blue>add link</font> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the LightCurve Object Class. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

### Index

1. [Instantiating the Ephemeris Objects](#section_1)
2. [Obtaining positions](#section_2)
3. [Ksi and Eta projection](#section_3)
4. [Adding offsets](#section_4)
5. [Pole Position Angle and Aperture Angle](#section_5)
6. [Apparent Magnitude](#section_5)

In [1]:
## import the Ephemeris Classes
from sora.ephem import EphemPlanete, EphemJPL, EphemKernel

## To facilitate, sora allows to import Ephem directly from the sora package.
from sora import EphemPlanete, EphemJPL, EphemKernel

<a id="section_1"></a>
### 1. Instantiating the Ephemeris Objects

As shown in the command above, there are 3 different types of Ephemeris Classes.

- EphemPlanete - Emulates what is done by fortran codes ephemplanete.f and fit_d2_ksi_eta.
- EphemJPL - Queries the JPL Horizons website and download ephemeris informations.
- EphemKernel - Reads the BSP files to calculate ephemeris.

**The Ephemeris Docstrings were designed to help the users. Also, each function has its Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

#### 1.1. Instantiating an EphemPlanete Object

In this case, a file must be given containing Julian Date, RA (in deg), DEC (in deg) and Distance (in AU), in this order, separated by an space. Please, look at the "ephem_phoebe_ph15_jd.txt" file for an example. Only the name of the object and the file with the ephemeris is required.

In [3]:
EphemPlanete?

In [2]:
eph_pla = EphemPlanete(name='Phoebe', ephem='input/ascii/ephem_phoebe_ph15_jd.txt')

In [3]:
print(eph_pla)

Ephemeris of Phoebe.
Radius: 106.0 km
Mass: 0.00e+00 kg

Ephem Planete:
Valid from 2019-06-07 02:54:00.000 until 2019-06-07 04:54:00.000
Ephem Error: RA*cosDEC: 0.050 arcsec; DEC: 0.050 arcsec



#### 1.2. Instantiating an EphemJPL Object

In this case, only the name of the object is required as this will be used to queries the JPL database. Another important keyword is the "id_type", which will be used to constrain the search of the object. For instance, if the name is "Europa" and the id_type is "smallbody" it looks the asteroid Europa, while if id_type is "majorbody" it looks for the Jovian satellite Europa. However, in some cases, there are still conflict name issue which solutions are being verified. A common solver is to use the IAU ID number in the name variable

In [6]:
EphemJPL?

In [4]:
eph_jpl = EphemJPL(name='Chariklo')

In [5]:
print(eph_jpl)

Ephemeris of Chariklo.
Radius: 121.0 km
Mass: 0.00e+00 kg

Ephemeris are downloaded from Horizons website



#### 1.3. Instantiating an EphemKernel Object

This is the most powerful Ephemeris Class. Since it reads the BSP files directly, it is faster. What is needed here is the name of the object, the kernel code which the object can be referred, and the list of kernels. Even if only one kernel is passed, it must be a list.

In [9]:
EphemKernel?

In [6]:
eph_ker = EphemKernel(name='Chariklo', code='2010199', kernels=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'])

In [7]:
print(eph_ker)

Ephemeris of Chariklo.
Radius: 121.0 km
Mass: 0.00e+00 kg

Ephem Kernel: CHARIKLO/DE438_SMALL (code=2010199)
Ephem Error: RA*cosDEC: 0.006 arcsec; DEC: 0.005 arcsec



#### 1.4 Other parameters

All the 3 Ephemeris Classes allow for the user to pass physical parameter, like radius, mass, the absolute magnitude (H), the Phase slope (G) and the ephemeris error in RA*cosDEC and DEC

If not given, the radius and ephemeris error will be downloaded from our online database.

In [8]:
eph_ker = EphemKernel(name='Chariklo', code='2010199', kernels=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'],
                      radius=75, mass=4.2e18, error_ra=0.02, error_dec=0.02)

In [9]:
print(eph_ker)

Ephemeris of Chariklo.
Radius: 75.0 km
Mass: 4.20e+18 kg

Ephem Kernel: CHARIKLO/DE438_SMALL (code=2010199)
Ephem Error: RA*cosDEC: 0.020 arcsec; DEC: 0.020 arcsec



### 2. Obtaining positions

The Ephemeris allows the user to get the on-sky geocentric position of the object for a certain time. This function does not exist for the EphemPlanete Class since, in this case, the ephemeris is an input parameter. The returned variable is an Astropy SkyCoord object, so their functions can be used.

In [14]:
eph_ker.get_position?

In [10]:
pos = eph_ker.get_position('2017-06-22 00:00:00')
print(pos)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, km)
    (283.86685123, -31.52237927, 2.19331444e+09)>


In [11]:
print(pos.ra)
print(pos.dec)
print(pos.distance.to('AU'))
print(pos.to_string('hmsdms', precision=5, sep=' '))

283d52m00.6644s
-31d31m20.5654s
14.661401433721714 AU
18 55 28.04429 -31 31 20.56536


In [12]:
eph_jpl.get_position('2017-06-22 00:00:00')

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, AU)
    (283.86685473, -31.52237985, 14.66140135)>

<a id="section_3"></a>
### 3. Ksi and Eta projection

Ephemeris objects calculates the orthographic projection (ksi and eta) of a site in the direction of a star given the following function. Ksi is in the East direction and Eta in the North direction.

The coordinates of the star given must be in the Geocentric Celestial Reference System (GCRS).

This is calculated automaticaly in Occultation.

In [17]:
eph_ker.get_ksi_eta?

Signature: eph_ker.get_ksi_eta(time, star)
Docstring:
Returns the on-sky position of the ephemeris relative to a star.

Parameters:
time (int, float):Time from which to calculate the position.
star (str, SkyCoord):The coordinate of the star in the same frame as the ephemeris.

Returns:
ksi, eta (float): on-sky position of the ephemeris relative to a star
File:      /mnt/homedir/occultation/SORA/sora/ephem.py
Type:      method


In [13]:
eph_ker.get_ksi_eta(time='2017-06-22 21:18:00', star='18 55 15.65250 -31 31 21.67051')

(1059.3876121640205, 522.2713452577591)

In [14]:
eph_jpl.get_ksi_eta(time='2017-06-22 21:18:00', star='18 55 15.65250 -31 31 21.67051')

(1173.7811108969795, 499.7081124443013)

**In the case of EphemPlanete, the time given must be in the range of the input ephemeris.**

In [15]:
print(eph_pla)

Ephemeris of Phoebe.
Radius: 106.0 km
Mass: 0.00e+00 kg

Ephem Planete:
Valid from 2019-06-07 02:54:00.000 until 2019-06-07 04:54:00.000
Ephem Error: RA*cosDEC: 0.050 arcsec; DEC: 0.050 arcsec



In [16]:
eph_pla.get_ksi_eta(time='2019-06-07 03:54:22.60', star='19 21 18.63201 -21 44 25.3924')

Fitting ephemeris position relative to star coordinate 19h21m18.632s -21d44m25.3924s
ksi = aksi*t² + bksi*t + cksi
eta = aeta*t² + beta*t + ceta
t=(jd-2458641.62083333)/(2458641.70416667-2458641.62083333)
        aksi=-74.33578570364584
        bksi=-120036.66964478238
        cksi=56728.18870944259
        aeta=-11.579369273364579
        beta=-17144.75884811901
        ceta=6972.329990726479
Residual RMS: ksi=0.004 km, eta=0.001 km


(-3685.746063311948, -1656.79625359059)

**For EphemPlanete, the user can also make the fit before running get_ksi_eta. If the fit is done, get_ksi_eta will only need the time param.**

In [22]:
eph_pla.fit_d2_ksi_eta?

Signature: eph_pla.fit_d2_ksi_eta(star, log=True)
Docstring:
Fits the on-sky ephemeris position relative to a star

Parameters:
    star (str, SkyCoord):The coordinate of the star in the same frame as the ephemeris.
    log (bool): if fit log is to printed. Default: True
File:      /mnt/homedir/occultation/SORA/sora/ephem.py
Type:      method


In [17]:
eph_pla.fit_d2_ksi_eta('19 21 18.63201 -21 44 25.3923')

Fitting ephemeris position relative to star coordinate 19h21m18.632s -21d44m25.3923s
ksi = aksi*t² + bksi*t + cksi
eta = aeta*t² + beta*t + ceta
t=(jd-2458641.62083333)/(2458641.70416667-2458641.62083333)
        aksi=-74.33578570364584
        bksi=-120036.66964478238
        cksi=56728.18870944259
        aeta=-11.579369458933602
        beta=-17144.7587881425
        ceta=6971.659735843644
Residual RMS: ksi=0.004 km, eta=0.001 km


In [18]:
eph_pla.get_ksi_eta(time='2019-06-07 03:54:22.60')

(-3685.746063311948, -1657.4664783438866)

<a id="section_4"></a>
### 4. Adding Offsets

If an ephemeris offset was obtained from astrometry, it can be added to the Ephemeris objects using add_offset. This values are given in $\Delta\alpha\cos\delta$ and $\Delta\delta$ in mas.

In [25]:
eph_ker.add_offset?

Signature: eph_ker.add_offset(da_cosdec, ddec)
Docstring:
Add an offset to the Ephemeris

Parameters:
    da_cosdec (int, float):Delta_alpha_cos_delta in mas
    ddec (int, float):Delta_delta in mas
File:      /mnt/homedir/occultation/SORA/sora/ephem.py
Type:      method


In [19]:
eph_ker.get_position('2017-06-22 00:00:00').to_string('hmsdms')

'18h55m28.0443s -31d31m20.5654s'

In [20]:
eph_ker.add_offset(40, 50)
eph_jpl.add_offset(40, 50)
eph_pla.add_offset(40, 50)

In [21]:
eph_ker.get_position('2017-06-22 00:00:00').to_string('hmsdms')

'18h55m28.0474s -31d31m20.5154s'

<a id="section_5"></a>
### 5. Pole Position Angle and Aperture Angle

For a given time, and given pole coordinate, EphemJPL and EphemKernel are able to calculate the Pole Position Angle relative to the North pole in the ICRS and the Aperture Angle of the object pole coordinate.

In [29]:
eph_ker.get_pole_position_angle?

Signature: eph_ker.get_pole_position_angle(pole, time)
Docstring:
Returns the geocentric position of the object.

Parameters:
pole (str, astropy.SkyCoord):Coordinate of the pole ICRS.
time (int, float): Time from which to calculate the position.

Returns:
position_angle (float): Position angle of the pole, in degrees
aperture_angle (float): Apeture angle of the pole, in degrees
File:      /mnt/homedir/occultation/SORA/sora/ephem.py
Type:      method


In [22]:
pos_ang, ap_ang = eph_ker.get_pole_position_angle('10 30 00 +45 50 00', '2017-06-22 00:00:00')
print(pos_ang)
print(ap_ang)

305.1787848766948 deg
46.652999109286604 deg


<a id="section_6"></a>
### 6. Apparent Magnitude

The Ephemeris objectcs calculates the Apparent Magnitude of an object using the H and G given by the user. The equation is:

$\phi_0 = \exp(-3.33(\tan(0.5 phase)^{0.63}))$

$\phi_1 = \exp(-1.87(\tan(0.5 phase)^{1.22}))$

$apmag = H - 2.5\log10((1-G)\phi_0 + G\phi_1) + 5\log10(sundist*dist)$

However, if these values are not given by the user, the apparent magnitude will be downloaded from JPL Horizons website. If JPL has H and G, this values will be saved and then used for calculation.

In [23]:
print(eph_ker.H, eph_ker.G)

nan nan


In [24]:
# downloading from JPL
eph_ker.apparent_magnitude('2017-06-22 00:00:00')

18.65

In [25]:
print(eph_ker.H, eph_ker.G)

6.7 0.15


In [26]:
# calculating using H and G
eph_ker.apparent_magnitude('2017-06-22 00:00:00')

18.649144367911177

**This Jupyter-Notebook was designed as a tutorial for how to work with the Ephemeris Object Classes. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

# The End